In [ ]:
import librosa, numpy as np
import pathlib as p
from matplotlib import pyplot as plt
import soundfile as sf
from ww_config.paths import *
from ww_config.config import *
import random
from scipy.signal import fftconvolve
import librosa.display
import os
import torch

In [ ]:
# some dir creations
if not RECORDINGS_PATH.exists():
  RECORDINGS_PATH.mkdir(parents=True, exist_ok=True)
WAKE_DIR.mkdir(parents=True, exist_ok=True)
WAKE_CHOPPED.mkdir(parents=True, exist_ok=True)
WAKE_AUGMENTED_DIR.mkdir(parents=True, exist_ok=True)

NON_WAKE_DIR.mkdir(parents=True, exist_ok=True)
NON_WAKE_CHOPPED_DIR.mkdir(parents=True, exist_ok=True)

### non-wake word data processing

In [ ]:
non_wake_mfcc = []
# no_of_plots = 30
sub_folder = NON_WAKE

chop_duration = 2   # every 2 seconds
file_count = 0
chop_length = chop_duration * SAMPLE_RATE   # no. of samples per chop
for file in p.Path.joinpath(RECORDINGS_PATH, sub_folder).glob("*"):
  y, sr = librosa.load(file, sr=SAMPLE_RATE)
 
  for i, chop_number in enumerate(range(int(y.shape[0]//chop_length))):
    filename = NON_WAKE_CHOPPED_DIR.joinpath(f"chopped_wake{file_count}.wav")
    sf.write(filename, y[i*chop_length:(i+1)*chop_length], sr)
    mfcc = librosa.feature.mfcc(y=y[i*chop_length:(i+1)*chop_length], sr=sr, n_mfcc=N_MFCC)
    non_wake_mfcc.append(mfcc)  
    file_count += 1
    print(f"Chopped original into multiples of {chop_duration} seconds and saved at {filename}")


In [ ]:
# fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 10))
librosa.display.specshow(non_wake_mfcc[0], sr=SAMPLE_RATE, x_axis='time')
plt.colorbar()
plt.title("MFCCs")
plt.xlabel("Time (s)")
plt.ylabel("MFCC Coefficients")
plt.show()

### wake word data pre-processing

In [ ]:
wake_mfcc = []
# no_of_plots = 30
sub_folder = WAKE
N_MFCC=20   # number of features

chop_duration = 2   # first 2 seconds only
i = 0
for file in p.Path.joinpath(RECORDINGS_PATH, sub_folder).glob("*"):
  y, sr = librosa.load(file, sr=SAMPLE_RATE)

  filename = WAKE_CHOPPED.joinpath(f"chopped_wake{i}.wav")
  sf.write(filename, y[:chop_length], sr)
  mfcc = librosa.feature.mfcc(y=y[:chop_length], sr=sr, n_mfcc=N_MFCC)
  wake_mfcc.append(mfcc)
  i += 1
  print(f"Chopped first {chop_duration} seconds and saved at {filename}")


In [ ]:
plt.figure(figsize=(12, 4))
librosa.display.specshow(wake_mfcc[1], sr=SAMPLE_RATE, x_axis='time')
plt.colorbar()
plt.title("MFCCs")
plt.xlabel("Time (s)")
plt.ylabel("MFCC Coefficients")
plt.show()

### Augmentation of wake data

In [ ]:
# We have to augment the chopped wake samples due to low in count
# Definig some Augmentation utility methods

def random_gain(y, min_db=-6, max_db=6):
  db = random.uniform(min_db, max_db)
  gain = 10.0 ** (db / 20.0)
  return y * gain

def add_background(y, bg, snr_db):
  # adjust background to desired snr relative to signal RMS
  rms_signal = np.sqrt(np.mean(y**2)) + 1e-9
  rms_bg = np.sqrt(np.mean(bg**2)) + 1e-9
  # compute linear scaling for bg
  snr_linear = 10 ** (snr_db / 20.0)
  scale = (rms_signal / (snr_linear * rms_bg))
  bg_scaled = bg * scale
  # trim or pad bg to match length
  if len(bg_scaled) >= len(y):
    start = random.randint(0, len(bg_scaled) - len(y))
    bg_segment = bg_scaled[start:start+len(y)]
  else:
    # repeat bg to fill
    repeats = int(np.ceil(len(y) / len(bg_scaled)))
    bg_tiled = np.tile(bg_scaled, repeats)[:len(y)]
    bg_segment = bg_tiled
  return y + bg_segment

def time_stretch(y, low=0.9, high=1.1):
  rate = random.uniform(low, high)
  # librosa time_stretch expects magnitude STFT -> use effects.time_stretch on waveform via resampling workaround:
  try:
    y_stretched = librosa.effects.time_stretch(y, rate)
  except Exception:
    # fallback: resample (this will change pitch)
    new_len = int(len(y) / rate)
    y_stretched = librosa.resample(y, orig_sr=SAMPLE_RATE, target_sr=int(SAMPLE_RATE*rate))
    y_stretched = librosa.resample(y_stretched, orig_sr=int(SAMPLE_RATE*rate), target_sr=SAMPLE_RATE)
  return y_stretched

def pitch_shift(y, semitones_low=-2, semitones_high=2):
  n_steps = random.uniform(semitones_low, semitones_high)
  return librosa.effects.pitch_shift(y, sr=SAMPLE_RATE, n_steps=n_steps)

def time_shift(y, max_shift_seconds=0.15):
  max_shift = int(max_shift_seconds * SAMPLE_RATE)
  shift = random.randint(-max_shift, max_shift)
  if shift == 0:
    return y
  if shift > 0:
    return np.pad(y, (shift, 0), mode='constant')[:len(y)]
  else:
    return np.pad(y, (0, -shift), mode='constant')[-shift:len(y)-shift]

def add_reverb(y, reverb_scale=0.3):
  # simple exponential impulse response
  ir_len = int(0.03 * SAMPLE_RATE)  # 30 ms IR
  t = np.arange(ir_len) / SAMPLE_RATE
  decay = np.exp(-t * (1.0 + random.random()*10.0))  # random decay rate
  ir = decay
  ir = ir / np.sum(np.abs(ir))
  y_rev = fftconvolve(y, ir)[:len(y)]
  return y * (1 - reverb_scale) + y_rev * reverb_scale

def random_filter(y):
  # light low-pass / high-pass via simple FFT mask
  # For simplicity use librosa.effects.preemphasis as a proxy
  if random.random() < 0.2:
    return librosa.effects.preemphasis(y)
  return y

In [ ]:

# load backgrounds into memory for speed
bg_files = [os.path.join(NON_WAKE_DIR, f) for f in os.listdir(NON_WAKE_DIR) if f.endswith(".wav")]
bg_sigs = []
for p in bg_files:
    bg, _ = librosa.load(p, sr=SAMPLE_RATE, mono=True)
    bg_sigs.append(bg)

wake_files = [os.path.join(WAKE_DIR, f) for f in os.listdir(WAKE_DIR) if f.endswith(".wav")]

file_count = 0
for wf in wake_files:
    y, _ = librosa.load(wf, sr=SAMPLE_RATE, mono=True)
    # optional: trim silence
    y, _ = librosa.effects.trim(y, top_db=30)
    for i in range(N_AUG_PER_FILE):
        y_aug = y.copy()
        # order: small random transforms
        if random.random() < 0.6:
            y_aug = time_stretch(y_aug, 0.92, 1.08)
        if random.random() < 0.6:
            y_aug = pitch_shift(y_aug, -1.5, 1.5)
        if random.random() < 0.7:
            y_aug = time_shift(y_aug, max_shift_seconds=0.12)
        if random.random() < 0.8:
            y_aug = random_gain(y_aug, -6, 6)
        if random.random() < 0.7 and len(bg_sigs) > 0:
            bg = random.choice(bg_sigs)
            snr = random.uniform(0, 20)  # 0 dB to 20 dB
            y_aug = add_background(y_aug, bg, snr_db=snr)
        if random.random() < 0.3:
            y_aug = add_reverb(y_aug, reverb_scale=random.uniform(0.05, 0.35))
        y_aug = random_filter(y_aug)
        # normalize to avoid clipping
        max_abs = np.max(np.abs(y_aug)) + 1e-9
        if max_abs > 1.0:
            y_aug = y_aug / max_abs * 0.99
        out_path = os.path.join(WAKE_AUGMENTED_DIR, f"aug_{file_count}.wav")
        sf.write(out_path, y_aug, SAMPLE_RATE)
        file_count += 1

print(f"Created {file_count} augmented files in {WAKE_AUGMENTED_DIR}")

### Forming the dataset

In [ ]:
def extract_features(file):
  y, _ = librosa.load(file, sr=SAMPLE_RATE)
  mfcc = librosa.feature.mfcc(y=y, sr=SAMPLE_RATE, n_mfcc=N_MFCC)
  mfcc = mfcc.T
  if len(mfcc) < MAX_LEN:
    pad = np.zeros((MAX_LEN - len(mfcc)), N_MFCC)
    mfcc = np.vstack([mfcc, pad])
  else:
    mfcc = mfcc[:MAX_LEN]
  return mfcc

# wake data
wake_data = []
for file in WAKE_AUGMENTED_DIR.glob("*"):
  wake_data.append(extract_features(file))

# non-wake data
non_wake_data = []
for file in NON_WAKE_CHOPPED_DIR.glob("*"):
  non_wake_data.append(extract_features(file))


class WakeWordDataset(torch.utils.data.Dataset):
  def __init__(self, X, y):
    self.X = [torch.tensor(x, dtype=torch.float32) for x in X]
    self.y = torch.tensor(y, dtype=torch.long)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, index):
    return self.X[index], self.y[index]

wake_word_dataset = WakeWordDataset(X=wake_data + non_wake_data, y=[1 for _ in range(len(wake_data))] + [0 for _ in range(len(non_wake_data))])

In [ ]:
print(len(wake_word_dataset), wake_word_dataset.X[0].shape)
print(wake_word_dataset.y.shape)

### Model architecture

In [ ]:
class WakeWord(torch.nn.Module):
  def __init__(self, input_size: int, hidden_size: int, output_size: int):
    super().__init__()
    self.lstm = torch.nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=1, batch_first=True)
    self.fc1 = torch.nn.Linear(in_features=hidden_size, out_features=output_size)

  def forward(self, input_data):
    lstm_output, _ = self.lstm(input_data)
    output = torch.nn.functional.sigmoid(self.fc1(lstm_output[:, -1, :]))
    return output

In [ ]:
model = WakeWord(input_size=N_MFCC, hidden_size=32, output_size=1)

In [ ]:
dataloader = torch.utils.data.DataLoader(dataset=wake_word_dataset, batch_size=10, num_workers=1)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
criterion = torch.nn.BCELoss()

### Training

In [ ]:
epochs = 25
loss_per_epoch = []

for epoch in range(epochs):
  train_loss_per_batch = 0
  model.train()
  for batch_count, (input_data, labels) in enumerate(dataloader):
    optimizer.zero_grad()
    outputs = model(input_data)

    loss = criterion(outputs, labels.view(-1, 1).float())
    loss.backward()
    optimizer.step()
    train_loss_per_batch += loss.item()
  loss_per_epoch.append(train_loss_per_batch/batch_count)
  print(f"Training loss per epoch: {train_loss_per_batch/batch_count}")


### Validating inferencing

In [33]:
mfcc = extract_features("/home/xd/Documents/wake-word/.data/val/wake.wav")
mfcc = torch.tensor(mfcc, dtype=torch.float32)
mfcc = mfcc.unsqueeze(0).float()

model.eval()
with torch.no_grad():
  prob = model(mfcc).item()

print(f"Wake word probability: {prob:.2f}")

Wake word probability: 0.64
